<a href="https://colab.research.google.com/github/fmanid/image-detexxxtion/blob/main/gaussian_blur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# See https://github.com/raghakot/keras-vis/issues/182
!pip install -I scipy==1.2.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.3/23.3 MB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scipy: filename=scipy-1.2.3-cp38-cp38-linux_x86_64.whl size=42810415 sha256=a6e1ab876130f9c8385bb469ec969949edbf26ff524cf46857a1eda52895d1e6
  Stored in directory: /root/.cache/pip/wheels/b8/f4/2a/2a6af3129d6d3f1fea59d476fd84326629e4075a73f5f35b1b
Successfully built scipy
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.4.0 requires scipy>=1.6, but you have scipy 1.2.3 which is incompatible.
pymc 4.1.4 requires scipy>=1.4.1, but you have scipy 1.2.3 which is incompatible.
plotnine 0.8.0 requires scipy>=1.5.0, but you have scipy 1.2.3 which is incompatible.
jaxlib 0.3.25+cuda11.cudnn805 requires scipy>=1.5, bu

In [2]:
# Clone the repositopry to get the MobileNet V1 model by DmitryM8
!git clone https://github.com/AIWintermuteAI/transfer_learning_sipeed.git

Cloning into 'transfer_learning_sipeed'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 63
Unpacking objects: 100% (66/66), done.


In [3]:
import keras
import numpy as np
from keras import backend as K
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense, GlobalAveragePooling2D, Dropout,Flatten
from keras.applications.mobilenet import preprocess_input
import sys
sys.path.append('/content/transfer_learning_sipeed')
from mobilenet_sipeed.mobilenet import MobileNet

In [4]:
#@title Dataset URL

dataset_url = 'https://www.dropbox.com/s/8k8x51cqeyql673/dataset_name.zip?dl=1' #@param {type:"string"}

In [5]:
dataset_root_dir = '/content/datasets'
dataset_tmp_filename = 'dataset.zip'

import os

# Make the directory for datasets if needed
if not os.path.isdir(dataset_root_dir):
  os.mkdir(dataset_root_dir)

# Download the dataset file from the URL as dataset.zip
get_ipython().system_raw('wget -O {} {}'.format(dataset_tmp_filename, dataset_url))

In [6]:
import zipfile

dataset_name = ''

with zipfile.ZipFile(dataset_tmp_filename) as zipfile:
    # Get the name of the root directory as the name of the dataset
    dataset_name = os.path.dirname(zipfile.namelist()[0]).split(os.sep)[0]
    zipfile.extractall(dataset_root_dir)

dataset_name

'dataset_name'

In [7]:
# Delete the dataset file
if os.path.isfile(dataset_tmp_filename):
  os.remove(dataset_tmp_filename)

In [8]:
IMAGE_SIZE = 224
ALPHA = 0.75
BATCH_SIZE = 32
EPOCHS = 20

In [9]:
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [10]:
dataset_dir = os.path.join(dataset_root_dir, dataset_name)

# Uncomment parameters to enable data augmentation
train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    # rotation_range=10,
                                    # width_shift_range=0.2,
                                    # height_shift_range=0.2,
                                    # shear_range=0.2,
                                    # zoom_range=0.3,
                                    # horizontal_flip=True,
                                    validation_split=0.1)

train_data = train_data_gen.flow_from_directory(dataset_dir,
                                                subset='training',
                                                target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                color_mode='rgb',
                                                batch_size=BATCH_SIZE,
                                                class_mode='categorical',
                                                shuffle=True)

validation_data = train_data_gen.flow_from_directory(dataset_dir,
                                                     subset='validation',
                                                     target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                     color_mode='rgb',
                                                     batch_size=BATCH_SIZE,
                                                     class_mode='categorical',
                                                     shuffle=True)

Found 32 images belonging to 3 classes.
Found 3 images belonging to 3 classes.


In [11]:
train_data.class_indices.items()

dict_items([('label_1', 0), ('label_2', 1), ('label_3', 2)])

In [12]:
# Write labels as labels.txt
with open('labels.txt', 'wt') as f:
    for key, value in train_data.class_indices.items():
        f.write(key + '\n')

In [17]:
# Using MobileNet V1
# See https://keras.io/applications/#mobilenet for details
base_model = MobileNet(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                       alpha=ALPHA,
                       include_top=False,
                       backend=keras.backend,
                       layers=keras.layers,
                       models=keras.models,
                       utils=keras.utils)

10626956/10626956 [==============================] - 0s 0us/step


In [32]:

import cv2
from os import listdir,makedirs
from os.path import isfile,join


path = r'/content/datasets/dataset_name/label_1' # Source Folder
dstpath = r'/content/desfolder1' # Destination Folder

try:
    makedirs(dstpath)
except:
    print ("Directory already exist, images will be written in asme folder")

# Folder won't used
files = [f for f in listdir(path) if isfile(join(path,f))] 

for image in files:
    try:
        img = cv2.imread(os.path.join(path,image))
        gray = cv.GaussianBlur(img,(5,5),0)
        dstPath = join(dstpath,image)
        cv2.imwrite(dstPath,gray)
    except:
        print ("{} is not converted".format(image))

Directory already exist, images will be written in asme folder
Creating_an_Image_Classification_Model_for_M5StickV_by_Transfer_Learning.ipynb.txt is not converted
